In [85]:
import robotic as ry
import numpy as np
import time

C = ry.Config()
C.addFile(ry.raiPath("../rai-robotModels/scenarios/pandaSingle.g"))
C.view()
target = C.addFrame("target", "table")

position = 9
target.setShape(ry.ST.marker, [.1]) \
  .setRelativePosition([ position, position , position ])

pos = target.getPosition()
cen = pos.copy()
C.view()


def IK( C, target, qHome ):
  q0 = C.getJointState()
  komo = ry.KOMO( C, 1, 10, 0, False )
  
  komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)          # objective to minimise collisions
  komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], q0)        # objective to minimise the cost of joint angles close to current joint state (q0) to minimise movement of robot
  komo.addObjective([], ry.FS.jointState, [], ry.OT.sos, [1e-1], qHome)     # objective to minimise the cost of joint angles close to home joint state (qHome) to minimise movement of robot
  komo.addObjective([], ry.FS.positionDiff, ["l_gripper", target], ry.OT.eq, [1e1])
  
  returnVal = ry.NLP_Solver(komo.nlp(), verbose=0).solve()
  
  return [ komo.getPath()[0], returnVal ] 

In [86]:
# first move the robot to the initial position
qHome = C.getJointState()

for t in range(20):
 time.sleep(.1)
 pos = cen + .98 * (pos-cen) + 0.02 * np.random.randn(3)
 target.setPosition(pos)
 q_target, ret = IK(C, "target", qHome)
 print(ret)
 C.setJointState(q_target)
 C.view()


{ time: 0.030199, evals: 191, done: 1, feasible: 0, sos: 1.07766, f: 0, ineq: 0, eq: 2515.09 }
{ time: 0.020582, evals: 134, done: 1, feasible: 0, sos: 0.538486, f: 0, ineq: 0, eq: 2520.18 }
{ time: 0.034968, evals: 150, done: 1, feasible: 0, sos: 0.539282, f: 0, ineq: 0, eq: 2523.59 }
{ time: 0.031972, evals: 167, done: 1, feasible: 0, sos: 0.539323, f: 0, ineq: 0, eq: 2521.92 }
{ time: 0.021979, evals: 131, done: 1, feasible: 0, sos: 0.537687, f: 0, ineq: 0, eq: 2521.71 }
{ time: 0.025681, evals: 131, done: 1, feasible: 0, sos: 0.536984, f: 0, ineq: 0, eq: 2519.25 }
{ time: 0.016894, evals: 131, done: 1, feasible: 0, sos: 0.538142, f: 0, ineq: 0, eq: 2521.66 }
{ time: 0.015192, evals: 139, done: 1, feasible: 0, sos: 0.539216, f: 0, ineq: 0, eq: 2519.39 }
{ time: 0.017074, evals: 131, done: 1, feasible: 0, sos: 0.539965, f: 0, ineq: 0, eq: 2523.93 }
{ time: 0.022009, evals: 142, done: 1, feasible: 0, sos: 0.540792, f: 0, ineq: 0, eq: 2518.66 }
{ time: 0.018376, evals: 131, done: 1, fe